# Clase 2: Librería Chainladder

<a href="https://colab.research.google.com/github/andesanalytics/python/blob/main/docs/modulo_4/Clase2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción a Chainladder

+ La biblioteca `chainladder` de Python es una herramienta utilizada en el análisis actuarial,
principalmente en el ámbito de seguros.

+ Su principal función es ayudar en la estimación de reservas de siniestros ocurridos y no reportados.

+ La librería posee multiples técnicas de proyección, que se basan en el análisis de triángulos de desarrollo de pérdidas, que
permiten proyectar los siniestros futuros basados en los datos históricos



## Instalación y Configuración

Para instalar la biblioteca `chainladder`, ejecuta el siguiente comando en tu entorno Python:


In [2]:
pip install chainladder

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00


Una vez instalada, puedes importarla y verificar su versión para asegurarte de que está correctamente instalada:

In [3]:
import chainladder as cl
print(cl.__version__)

0.8.18


## Manejo de Datos: Propiedades Básicas
+ El paquete chainladder tiene su propia estructura de datos `:class:Triangle` que se comporta de manera muy similar a un `DataFrame` de ``pandas``
+ Por qué no ``pandas``? Principalmente porque podemos manejar multiples triángulos al mismo tiempo, junto con la eficiencia en memoria
+ La estructura `Triangle` es una estructura de datos en 4D con ejes etiquetados. Estos ejes son ``index``, ``columns``, ``origin``, ``development``


``index`` (axis 0): Es la agrupación mínima a la cual se requiera ver los datos (por ejemplo por producto, tipo de negocio, ramo fecu). Tal como en ``pandas.multiIndex``, se pueden tener más de una columa dentro de `index`

``columns`` (axis 1): Son el o los valores numéricos que deseas agrupar en triángulo. Si bien el valor obvio a acumular es el de siniestros pagados, también puedes tener otros triángulos como el número de siniestros, o los siniestros pendientes por ejemplo.

``origin`` (axis 2): Es el eje vertical dentro de un triángulo tradicional. Corresponde al mes, trimestre, semestre o año en el cual ocurrió el siniestro. La periodicidad se puede definir por el usuario.

``development`` (axis 3): Es el eje horizontal del triángulo. Representa el periodo en que el siniestro fue reportado. Al igual que el eje `origin` se puede tomar la periodicidad mensual, trimestral, semestral, anual.

### ¿Cómo se trabaja con una clase ``Triangle``?
+ Independiente de poseer 4 dimensiones, esta clase permite que se interactúe con él de la misma manera que en `pandas`. Puedes usar los ejes `index` y `columns` de la misma manera que en un `DataFrame`, donde cada elemento será un triángulo tradicional. La siguiente imagen muestra como sería la estructura de la clase.




### ¿Cómo se construye una clase ``Triangle``?

+ Una clase `Triangle` se crea a partir de un `DataFrame` de `pandas`, el cuál debe poseer como mínimo las siguientes columnas.
    * Dos columnas de tipo fecha que representen los ejes ``origin`` y ``development``.
    * Una columna numérica que represente el eje ``columns``



In [ ]:
import pandas as pd
raa_df = pd.read_csv("https://raw.githubusercontent.com/casact/chainladder-python/master/chainladder/utils/data/raa.csv")
raa_df.head(10)

Vemos que dataframe cargado posee las columnas mínimas requeridas.
La forma para obtener una clase de tipo `Triangle` se obtiene con el método `Triangle`

In [ ]:
raa = cl.Triangle(raa_df,origin="origin",development="development",columns="values",cumulative=True)
print(raa.shape)
raa

+ Como se puede observar, la información fue agrupada como triángulo.
+ Las dimensiones del triángulo indican 4 dimensiones, que corresponden a los ejes anteriormente comentados
+ La visualización del triángulo cambiará si nuestra data contiene más de un valor en el eje ``columns`` o más de un valor en el eje `index`

### ``Triangle`` en 4D
Ahora revisemos el siguiente triangulo cargado de los datos de muestra de la librería

In [ ]:
triangle = cl.load_sample('CLRD')
print(triangle.shape)
triangle

Vemos que la forma de mostrar la data cambia, debido a que el eje ``index`` posee 775 diferentes niveles, mientras que el eje `columns` posee 6 distintas columnas para formar triángulos.

### Parametro `cumulative`
+ El parámetro ``cumulative`` indica si los datos que entregamos en el eje `columns` corresponden a datos acumulativos a la fecha o son los datos específicos de la fecha.
+ Es completamente opcional. Si no se especifica, el Triángulo inferirá su estado ``cumulative/incremental`` en el punto en el que llame a los métodos ``cum_to_incr`` o ``incr_to_cum`` (que sirven para pasar de un estado a otro)
+ El método `is_cumulative` nos permite saber si un triangulo tiene su parametro `cumulative` en `True` o `False`

In [ ]:
raa.cum_to_inc()

### Valoración vs Desarrollo
Si bien la mayoría de los estimadores que utilizan triángulos esperan que el período de desarrollo se exprese como una edad sobre el periodo de origen (diagonal de abajo hacia arriba), es posible transformar un triángulo en un triángulo de valoración, donde los períodos de desarrollo se convierten en períodos de valoración (es decir con la diagonal de arriba hacia abajo) con el método `dev_to_val`

Los triangulos poseen también la propiedad `is_val_tri` que indica si el triangulo está en modo valoración (diagonal de arriba hacia abajo)

raa.dev_to_val()
raa.is_val_tri

### Granularidad del Triangulo
Si el triángulo posee en sus ejes `origin` o `development` periodicidades más frecuentes a una anual (mensual, trimestral), se puede facilmente pasar a una granularidad mayor con el metodo `grain`. Este método reconoce las siguientes granularidades.
+ Anual: Yearly (Y)
+ Trimestral: Quarterly (Q)
+ Mensual: Monthly (M)

In [ ]:
# CARGAMOS DATA DE PRUEBA CON LA FUNCION load_sample()
df_quarterly=cl.load_sample('quarterly')
df_quarterly
df_quarterly.grain('OYDY')

### Link Ratios
Los factores de desarrollo por cada periodo origen-desarrollo (también conocidos en inglés como *link ratios*) pueden ser visualizados con la propiedad ``link_ratio``. Los triángulos además tienen un método llamado ``heatmap`` que puedes llamar opcionalmente para darle formatos a las salidas del triángulo. Este metodo requiere de IPython/Jupyter notebook para ejecutarse.

In [ ]:
triangle_abc = cl.load_sample('abc')
triangle_abc
triangle_abc.link_ratio
triangle_abc.link_ratio.heatmap()

## Sintaxis Tipo Pandas
### Filtros y Slicing
Podemos filtrar y seleccionar triángulos de menor tamaño al igual que en pandas. Funciones como `iloc` o `loc` aplican de la misma manera.

**Recordar que bajo la lógica de esta librería el eje `index` vendría siendo el eje de filas y el eje `columns` el de columnas en pandas.**

In [ ]:
# Cargamos data con load_sample
clrd = cl.load_sample('clrd')
# Slicing con iloc
clrd.iloc[0,1]
# Filtros del eje ``index`` mediante una condicion
clrd[clrd['LOB']=='othliab']
# Filtro del eje `columns` mediante una selecion de columnas
clrd['EarnedPremDIR']

### Virtual Columns
Hay casos en los que queremos posponer los cálculos, podemos crear columnas "virtuales" que pospongan los cálculos cuando sea necesario. Estas columnas se pueden crear envolviendo una columna normal en una función. Las expresiones lambda funcionan como una representación ordenada de columnas virtuales.

In [ ]:
clrd = cl.load_sample('clrd')
# A physical column with immediate evaluation
clrd['PaidLossRatio'] = clrd['CumPaidLoss'] / clrd['EarnedPremDIR']
clrd.sum()['PaidLossRatio']
# A virtual column with deferred evaluation
clrd['PaidLossRatio'] = lambda clrd : clrd['CumPaidLoss'] / clrd['EarnedPremDIR']
# Good - Defer loss ratio calculation until after summing premiums and losses
clrd.sum()['PaidLossRatio']

### Agregaciones
Al igual que los pandas, puedes agregar múltiples triángulos dentro de un Triángulo usando ``sum()`` que opcionalmente puede combinarse con ``groupby()``.

In [ ]:
clrd = cl.load_sample('clrd')
clrd.sum()
clrd.groupby('LOB').sum()

* Por defecto, la agregación se aplicará al primer eje con una longitud mayor que 1.
* Alternativamente, puede especificar el eje usando el argumento ``axis`` del método agregado.

### Convertir a ``DataFrame``
Cuando un triángulo se presenta con un único nivel del eje ``index`` y una sola columna (eje `column`), se convierte en un objeto 2D (En un triángulo como vimos al comienzo). Como tal, su formato de visualización cambia a uno similar a un marco de datos. Estos triángulos 2D se pueden convertir fácilmente en un marco de datos de pandas utilizando el método ``to_frame``. A partir de este punto, los resultados se pueden operar directamente en pandas.

La funcionalidad ``to_frame`` funciona cuando un Triángulo se divide en dos ejes cualesquiera y no se limita solo a `index` y `column`

In [ ]:
clrd = cl.load_sample('clrd')
clrd
clrd[clrd['LOB']=='ppauto']['CumPaidLoss'].sum().to_frame()
clrd['CumPaidLoss'].groupby('LOB').sum().latest_diagonal.to_frame()



## Modelado Actuarial Básico

El modelo Chainladder básico se puede aplicar de la siguiente manera:

```python
modelo = cl.Chainladder()
resultado = modelo.fit(triangulo)
print(resultado)
```

Este ejemplo muestra cómo ajustar el modelo Chainladder básico a un triángulo de desarrollo de pérdidas.



## Métodos Avanzados de Modelado

Además del modelo Chainladder básico, la biblioteca ofrece otros modelos como Bornhuetter-Ferguson.
Aquí hay un ejemplo de cómo aplicarlo:

```python
modelo_bf = cl.BornhuetterFerguson()
resultado_bf = modelo_bf.fit(triangulo)
print(resultado_bf)
```

Este modelo combina los datos históricos con a prioris para hacer proyecciones.



## Visualización y Reporte

`chainladder` facilita la visualización de los resultados. Por ejemplo, puedes graficar los resultados
del modelo:

```python
import matplotlib.pyplot as plt

# Graficar los resultados
triangulo.plot()
plt.show()
```

Este código genera un gráfico del triángulo de desarrollo de pérdidas.



## Ejemplos y Casos Prácticos

Aquí puedes incluir varios ejemplos prácticos y casos de estudio utilizando conjuntos de datos reales
o simulados para ilustrar cómo se puede aplicar `chainladder` en situaciones del mundo real.



## Recursos Adicionales y Conclusión

Para aprender más sobre `chainladder` y análisis actuarial, puedes consultar la documentación oficial
y otros recursos en línea. Siempre es útil mantenerse al día con las últimas tendencias y técnicas en
el campo actuarial.

Conclusión: `chainladder` es una herramienta esencial para los actuarios en el ámbito de seguros,
ofreciendo un enfoque robusto y flexible para la estimación de reservas.
